In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from config import props, filenames
import analysis
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
SAVE_FIGS = False

In [4]:
data_raw = pd.read_pickle(filenames.data+'data.pkl')
data_raw.t = data_raw.t - data_raw.t[0]
data_convoluted = analysis.data_cleaning(data_raw.copy())
# data_convoluted.I=gaussian_filter1d(data_convoluted.I, 100)
pert_times = analysis.find_perts(data_convoluted.copy())
data = analysis.data_interpolation(data_convoluted.copy(), pert_times)

In [ ]:
data.I = gaussian_filter1d(data.I, 10)

In [ ]:
data = analysis.data_interpolation(data_convoluted.copy(), pert_times)

In [ ]:
plt.plot(np.diff(data_convoluted.I, n=2, prepend=0, append=0))

In [ ]:
# mpl.use('pgf')
cut_data = data_convoluted[(data_convoluted.t>8250) & (data_convoluted.t < 8400)]
fig = plt.figure(figsize=[3.2,2.4])
plt.plot(cut_data.t, cut_data.I)
plt.title('High Amplitude oscillation, 6V')
plt.xlabel('Time [s]')
plt.ylabel(r'J [mA/cm$^2$]')
plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
plt.show()

In [ ]:
plt.figure(figsize=[3.2,2.4])
plt.plot(cut_data.I, cut_data.emsi)
plt.xlabel(r'J [mA/cm$^2$]')
plt.ylabel(r'$\xi$ [\%]')
plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
plt.show()

In [ ]:
plt.figure(figsize=(2.8, 2.1))
plt.xlim(-10,210)
plt.axvline(31.96, c='red', ls='--')
plt.plot(data_raw.t, data_raw.I*100/props.el_area)
plt.xlabel('Time [s]')
plt.ylabel('J [mA/cm^2]')
plt.title('b) HA Initialisation')
plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)

In [ ]:
plt.figure(figsize=(2.8, 2.1))
plt.xlim(9660, 9990)
plt.plot(data.t, data_convoluted.I)
for det_point in cycles.start:
    if det_point > 9600 and det_point < 10000:
        plt.axvline(det_point, c='gray', ls=':')
        plt.scatter(det_point, data_convoluted.I[data_convoluted.t == det_point], c='r', marker='x', zorder=10)
plt.xlabel('Time [s]')
plt.ylabel(r'J [mA/cm$^2$]')
plt.title('Mean-current Crossings')
plt.axhline(data.I.mean()*1, c='y', ls='--')
plt.subplots_adjust(top=0.882,
bottom=0.218,
left=0.212,
right=0.981,
hspace=0.1,
wspace=0.1)

In [ ]:
data_cut = data_convoluted[(data.t > 9660) & (data.t < 9990)]
fig, ax = plt.subplots(figsize=(2.8, 2.1))
plt.plot(data_cut.I, data_cut.emsi)
plt.plot([0, 1], [0, 1], c='gray', ls='-.', transform=ax.transAxes)
plt.plot([0, 1], [1, 0], c='gray', ls='-.', transform=ax.transAxes)
plt.plot([.5, .5], [0, 1], c='gray', ls='-.', transform=ax.transAxes)
plt.plot([0, 1], [.5, .5], c='gray', ls='-.', transform=ax.transAxes)
plt.scatter(0.5, 0.883, transform=ax.transAxes, c='r', marker='x', zorder=10, s=100)
plt.xlabel(r'J [mA/cm$^2$]')
plt.ylabel(r'$\xi$ [%]')
plt.title('2D Phase Determination')
plt.subplots_adjust(top=0.882,
bottom=0.218,
left=0.212,
right=0.981,
hspace=0.1,
wspace=0.1)

In [5]:
plt.figure()
plt.plot(data_raw.t, data_raw.I, label='raw')
plt.scatter(pert_times, data_raw.loc[data_raw.t.isin(pert_times), 'I'], marker='x', c='r')
plt.plot(data.t, data.I, label='treated')
plt.legend(loc=1)
plt.show()
if SAVE_FIGS:
    plt.savefig(filenames.notes+'current_waveform.png')

In [5]:
cycles = analysis.find_cycles(data, pert_times)

In [6]:
perts = analysis.pert_response(data, cycles, pert_times[:-1])

In [7]:
# fig, axs = plt.subplots(2, sharex=True, figsize=(2.8, 4.1))
fig, axs = plt.subplots(2, sharex=True)

axs[0].plot(cycles['start'], cycles['duration'])
axs[0].plot(perts.time, perts.expected_period)
axs[0].set_ylabel('Period [s]')
# axs[0].scatter(cycles['start'][cycles['had_pert']], cycles['duration'][cycles['had_pert']], c='r', s=10)

axs[1].scatter(perts['time'], perts['phase'], marker = 'x')
axs[1].plot(perts['time'], perts['phase'])
axs[1].set_ylabel(r'$\phi$ at Perturbation')

fig.suptitle('b) Randomised Intervals')
fig.supxlabel('Time [s]')
# fig.subplots_adjust(top=0.918,bottom=0.12,left=0.232,right=0.971,hspace=0.147,wspace=0.2)
if SAVE_FIGS:
    fig.savefig(filenames.notes+'period_vs_time.png')

In [27]:
# fig, axs = plt.subplots(2, sharex=True, figsize=(2.8, 4.1))
fig, axs = plt.subplots(1, figsize=(5.6, 3), sharex=True)

axs.plot(cycles['start'], cycles['duration'], label='Measured Period')
axs.plot(perts.time, perts.expected_period, label = 'Expected Period')
axs.set_ylabel('Period [s]')
plt.legend()
# axs[0].scatter(cycles['start'][cycles['had_pert']], cycles['duration'][cycles['had_pert']], c='r', s=10)

axs.set_xlabel('Time [s]')

# data_cut= data_convoluted[(data.t > 28500) & (data.t < 34900)]
# axs[0].set_xlim(30100, 34700)
# axs[1].plot(data_convoluted.t, data_convoluted.I)
# axs[1].set_ylabel(r'J [mA/cm$^2$]')
# for t in pert_times:
#     plt.scatter(t, data[data.t == t].I, marker='x', c='r', zorder=10)

fig.suptitle('Period versus Time')
fig.tight_layout()
# fig.supxlabel('Time [s]')
# fig.subplots_adjust(top=0.918,bottom=0.12,left=0.232,right=0.971,hspace=0.147,wspace=0.2)
if SAVE_FIGS:
    fig.savefig(filenames.notes+'period_vs_time.png')

In [15]:
def on_pick(event) -> tuple[mpl.figure.Figure, mpl.figure.Axes]:
    ax_phase.cla()
    ax_current.cla()

    pert_time = perts.time.iloc[event.ind[0]]
    print(pert_time)
    data_before = data_convoluted[(data.t > pert_time - props.max_period) & (data.t < pert_time)]
    data_after = data_convoluted[(data.t > pert_time) & (data.t < pert_time + 2*props.max_period)]

    ax_phase.plot(data_before.I, data_before.emsi)
    ax_phase.plot(data_after.I, data_after.emsi)
    ax_phase.scatter(data_before.I.iloc[-1], data_before.emsi.iloc[-1], c='r')
    fig_phase.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- phase space')
    fig_phase.supxlabel(r'current [mA/cm$^2$]')
    fig_phase.supylabel(r'emsi signal [$\xi$]')
    fig_phase.canvas.draw()

    ax_current.plot(data_before.t, data_before.I)
    ax_current.plot(data_after.t, data_after.I)
    ax_current.scatter(data_before.t.iloc[-1], data_before.I.iloc[-1], c='r')
    fig_current.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- current')
    fig_current.supxlabel('time [s]')
    fig_current.supylabel(r'current [mA/cm$^2$]')
    fig_current.canvas.draw()

In [8]:
if props.pert_type == 'U':
    title = f'PRC for {props.osc_type} {props.voltage}{'+' if props.pert_strength>0 else ''}{props.pert_strength}V, {props.pert_dt}s'
else:
    title = f'PRC for {props.voltage}V {props.osc_type}, {props.pert_dt}s {'+' if props.pert_strength>0 else '-'}ve light'
title = r'b) Negative Light Perturbation'
fig, ax = plt.subplots(figsize=(2.8, 2.1))
# fig, ax = plt.subplots(figsize=(5.6, 4.2))
fig.suptitle(title)
ax.set_xlabel(r'Phase $\phi$')
ax.set_ylabel('Phase Resetting Z')
ax.axhline(0, ls='--', c='gray')
ax.scatter(perts['phase'], perts['response'], c=perts['time'], picker=True, s=10)
# plt.gca().invert_yaxis()
# ax.set_ylim(-0.035, 0.0133)

fig.subplots_adjust(top=0.88,bottom=0.22,left=0.27,right=0.97,hspace=0.2,wspace=0.2)
# fig.tight_layout()

# fig_phase, ax_phase = plt.subplots()
# fig_current, ax_current = plt.subplots() 
# id = fig.canvas.mpl_connect('pick_event', on_pick)
# if SAVE_figS:
#     fig.save_fig(filenames.notes+'prc.png')

In [ ]:
fig.canvas.mpl_disconnect(14)

In [8]:
fig, axs = plt.subplots(2, sharex=True)
fig.suptitle(title)
axs[0].axhline(0)
axs[0].scatter(perts['phase'], perts['response'], c=perts['time'])
axs[0].set_ylabel('Phase Response')
one_period = data_convoluted[(data_convoluted.t>cycles.loc[100, 'start']) & (data_convoluted.t<cycles.loc[101, 'start'])]
axs[1].plot((one_period.t-cycles.start[100])/cycles.loc[100, 'duration'], one_period.I/props.el_area*100)
axs[1].set_ylabel(r'J [mA/cm$^2$]')
plt.show()

In [ ]:
id = fig.canvas.mpl_connect('pick_event', on_pick)

In [12]:
fig, axs = plt.subplots(2,2, sharex = True, sharey=True, figsize = (5.6, 4.2))
things_to_plot = ['response_0', 'response_1', 'response_2', 'response_3']
names = iter(['1st Response', '2nd Response', '3rd Response', '4th Response'])
for (thing, ax) in zip(things_to_plot, axs.flatten()):
    ax.scatter(perts['phase'], perts[thing], c=perts['time'], s=10)
    ax.set_title(next(names))
    ax.axhline(0, ls='--')
fig.suptitle(title)
fig.supxlabel(r'$\phi$')
fig.supylabel(r'$\Delta\phi$')
# fig.tight_layout()
fig.subplots_adjust(top=0.873,bottom=0.12,left=0.106,right=0.963,hspace=0.3,wspace=0.1)
if SAVE_FIGS:
    fig.savefig(filenames.notes+'prc_full.png')